In [44]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [39]:
cliques_df = pd.read_csv("cliques2versions.tsv",sep = '\t')
test_ids = np.load("test_ids.npy")
train_cliques = np.load("train_cliques.npy")
val_cliques = np.load("val_cliques.npy")
cliques_df

,clique,versions
0,39475,"[343223, 361210, 114472, 134744, 271362, 30747..."
1,20077,"[343224, 350590, 170706, 266043, 314556, 30764..."
2,22290,"[343225, 343986, 344624, 345116, 345312, 33796..."
3,17098,"[343226, 220430]"
4,26349,"[343227, 286579]"
...,...,...
41611,16898,"[70624, 76088]"
41612,31616,"[70632, 76025]"
41613,40137,"[71481, 73797]"
41614,21450,"[71855, 74678]"


In [38]:
print(test_ids.shape)
print(train_cliques.shape)
print(val_cliques.shape)

(55170,)
(39535,)
(2081,)


In [41]:
train_df = pd.DataFrame(columns = ["clique","versions"])
val_df = pd.DataFrame(columns = ["clique","versions"])
for i in range(len(cliques_df)):
    if cliques_df.iloc[i,0] in train_cliques:
        train_df.loc[len(train_df)] = cliques_df.iloc[i,:]
    else:
        val_df.loc[len(val_df)] = cliques_df.iloc[i,:]

In [57]:
def create_data(df,root):
    tmp_df = pd.DataFrame(columns = ["starts"])
    tmp_df.loc[0,0] = 0
    file_names = []

    for i in tqdm(range(len(df))):
        lst = eval(df.iloc[i,1])
        file_names += lst
        if i != len(df):
            tmp_df.loc[i+1,0] = tmp_df.loc[i,0] + len(lst)

    x = np.zeros((len(file_names),84,50))

    for i in tqdm(range(len(file_names))):
        name = file_names[i]
        x[i] = np.load(Path(root) / str(name%1000//100) / str(name%100//10) / str(name%10) / (str(name) + ".npy"))
    
    return tmp_df, x.astype("float32")

In [58]:
train_starts, train_data = create_data(train_df,"train")
val_starts, val_data = create_data(val_df,"train")

100%|██████████| 15885/15885 [00:18<00:00, 857.18it/s] 


In [59]:
train_starts.to_csv("train_starts.csv")
np.save("train_data.npy",train_data)
val_starts.to_csv("val_starts.csv")
np.save("val_data.npy",val_data)

In [60]:
test_data = np.zeros((len(test_ids),84,50))

for i in tqdm(range(len(test_ids))):
    name = test_ids[i]
    test_data[i] = np.load(Path("test") / str(name%1000//100) / str(name%100//10) / str(name%10) / (str(name) + ".npy"))

test_data = test_data.astype("float32")

100%|██████████| 55170/55170 [00:33<00:00, 1637.67it/s]


In [61]:
np.save("test_data.npy",test_data)